In [11]:
# In this serie of the aim is to classify the dogs' breed. The dataset exists in: https://www.kaggle.com/competitions/dog-breed-identification/data
# A naive CNN model, a transfer learning model, and concatenation of different transfer learning models are implemented here.

# Importing dependencies

In [1]:
import tensorflow as tf
import os, shutil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from resizeimage import resizeimage

# Reading data

In [2]:
main_dir = "directory_to_the_dataset_folder"
train_dir = os.path.join(main_dir, 'train')
images_name = os.listdir(train_dir)
df = pd.read_csv (os.path.join(main_dir, 'labels.csv'))


In [3]:
# Finding identical labels
breeds = []
for breed in df['breed']:
    if breed not in breeds:
        breeds.append(breed)

In [4]:
# Creating label map
label_map ={}
for i, breed in enumerate(breeds):
    label_map[breed] = i

In [5]:
#creating folder for train
copied_train_data_dir = 'directory_to_train_data'
for breed in breeds:
    new_dir = os.path.join(copied_train_data_dir, breed)
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)

In [6]:
#creating folder for validation
copied_train_data_dir = 'directory_to_validation_data'
for breed in breeds:
    new_dir = os.path.join(copied_train_data_dir, breed)
    if not os.path.exists(new_dir):
        os.mkdir(new_dir)

In [7]:
# defining the number of training samples for each class
train_samples_each_class = 40
# copying train classes in their folders
copy_dir = 'train_data_directory'
val_dir = 'validation_data_directory'
counter = np.zeros((np.array(breeds).shape[0]))
for i, breed in  enumerate(df['breed']):
    for target_breed in breeds:
        if breed == target_breed:
            counter[label_map[breed]] += 1
            img_name = df['id'][i] + '.jpg'
            img_dir = os.path.join(train_dir, img_name)
            if counter[label_map[breed]] < train_samples_each_class:
                dst = os.path.join(copy_dir, breed)
                dst = os.path.join(dst, img_name)
                shutil.copy(img_dir, dst)
            else:
                dst = os.path.join(val_dir, breed)
                dst = os.path.join(dst, img_name)
                shutil.copy(img_dir, dst)
        
    

# 1- Creating naive model

In [ ]:
# importing libraries
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# creating a naive CNN model
model = Sequential()

model.add(layers.Conv2D(32, 3, activation = 'relu', input_shape = (150, 150, 3)))
model.add(layers.MaxPooling2D(2))


model.add(layers.Conv2D(64, 3, activation = 'relu'))
model.add(layers.MaxPooling2D(2))


model.add(layers.Conv2D(128, 3, activation = 'relu'))
model.add(layers.MaxPooling2D(2))


model.add(layers.Conv2D(128, 3, activation = 'relu'))
model.add(layers.MaxPooling2D(2))
    
    
model.add(layers.Flatten())

model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(np.array(breeds).shape[0], activation = 'softmax'))

model.compile(optimizer = 'rmsprop',
              loss = 'categorical_crossentropy',
              metrics = ['acc'])

train_datagen = ImageDataGenerator(rescale = 1./255) 
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(copy_dir,
                                                    target_size = (150, 150),
                                                    batch_size = 20,
                                                    class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size = (150, 150),
                                                        batch_size = 20,
                                                        class_mode = 'categorical')

In [ ]:
# training
history = model.fit_generator(
    train_generator,
    steps_per_epoch=40,
    epochs=80,
    validation_data=validation_generator,
    validation_steps=50)

# 2- Transfer Learning

In [8]:
# In this part, the model is created using transfered information from the other existing models.
# Some of the models that have better performance of this dataset is imported here:
from tensorflow.keras.applications import InceptionResNetV2, Xception, NASNetMobile, MobileNetV2, DenseNet121, DenseNet169
# importing libraries
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# using a data generator to preprocess and augment the existing data.
train_datagen = ImageDataGenerator(rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest') 
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(copy_dir,
                                                    target_size = (224, 224),
                                                    batch_size = 20,
                                                    class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size = (224, 224),
                                                        batch_size = 20,
                                                        class_mode = 'categorical')
# Selecting a model for the basis of the transfer learning. You can change it to any existing models.
Chosen_model = NASNetMobile
conv_base = Chosen_model(weights = 'imagenet',
                  include_top = False,
                  input_shape = (224, 224, 3))
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(np.array(breeds).shape[0], activation = 'softmax'))
model.summary()
conv_base.trainable = False



optimizer = tf.keras.optimizers.Adam(0.0001)


model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['acc'])


history = model.fit_generator(
    train_generator,
    steps_per_epoch=40,
    epochs=80,
    validation_data=validation_generator,
    validation_steps=50)

Found 4680 images belonging to 120 classes.
Found 5542 images belonging to 120 classes.
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 flatten (Flatten)           (None, 51744)             0         
                                                                 
 dense (Dense)               (None, 512)               26493440  
                                                                 
 dropout (Dropout)           (None, 512)               0         
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 120)               30840     
                                  

C:\Users\Mohsen\AppData\Local\Temp\ipykernel_17056\1432202669.py:54: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/80
40/40 [==============================] - 29s 474ms/step - loss: 4.5143 - acc: 0.1050 - val_loss: 3.3399 - val_acc: 0.3270
Epoch 2/80
40/40 [==============================] - 15s 382ms/step - loss: 3.2540 - acc: 0.2988 - val_loss: 2.1013 - val_acc: 0.5230
Epoch 3/80
40/40 [==============================] - 15s 369ms/step - loss: 2.5515 - acc: 0.4387 - val_loss: 1.6191 - val_acc: 0.6220
Epoch 4/80
40/40 [==============================] - 14s 350ms/step - loss: 2.3404 - acc: 0.4663 - val_loss: 1.1997 - val_acc: 0.7000
Epoch 5/80
40/40 [==============================] - 13s 339ms/step - loss: 1.9680 - acc: 0.5300 - val_loss: 1.1871 - val_acc: 0.6930
Epoch 6/80
40/40 [==============================] - 13s 328ms/step - loss: 1.8015 - acc: 0.5575 - val_loss: 1.1743 - val_acc: 0.7170
Epoch 7/80
40/40 [==============================] - 13s 316ms/step - loss: 1.7214 - acc: 0.5788 - val_loss: 0.9954 - val_acc: 0.7300
Epoch 8/80
40/40 [==============================] - 12s 303ms/step - 

40/40 [==============================] - 11s 266ms/step - loss: 0.8851 - acc: 0.7500 - val_loss: 1.0759 - val_acc: 0.7100
Epoch 63/80
40/40 [==============================] - 10s 263ms/step - loss: 0.9529 - acc: 0.7475 - val_loss: 0.9559 - val_acc: 0.7640
Epoch 64/80
40/40 [==============================] - 10s 260ms/step - loss: 0.9923 - acc: 0.7250 - val_loss: 0.9079 - val_acc: 0.7650
Epoch 65/80
40/40 [==============================] - 11s 267ms/step - loss: 0.9458 - acc: 0.7312 - val_loss: 1.0052 - val_acc: 0.7500
Epoch 66/80
40/40 [==============================] - 10s 262ms/step - loss: 1.0020 - acc: 0.7375 - val_loss: 0.9371 - val_acc: 0.7480
Epoch 67/80
40/40 [==============================] - 10s 259ms/step - loss: 0.8911 - acc: 0.7475 - val_loss: 0.9140 - val_acc: 0.7790
Epoch 68/80
40/40 [==============================] - 11s 265ms/step - loss: 0.9004 - acc: 0.7450 - val_loss: 0.9422 - val_acc: 0.7550
Epoch 69/80
40/40 [==============================] - 10s 257ms/step - loss

# 3- Contactenating models

In [10]:
# This model concatenates the extracted features from two basis model and classifies based on these two models' features.

from keras.callbacks import LearningRateScheduler
from keras.models import Model
from tensorflow.keras.applications import InceptionResNetV2, Xception, NASNetMobile, MobileNetV2, DenseNet121, DenseNet169
import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Conv2D, MaxPool2D,  \
    Dropout, Dense, Input, concatenate,      \
    GlobalAveragePooling2D, AveragePooling2D,\
    Flatten

train_datagen = ImageDataGenerator(rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest') 
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(copy_dir,
                                                    target_size = (350, 350),
                                                    batch_size = 16,
                                                    class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size = (350, 350),
                                                        batch_size = 16,
                                                        class_mode = 'categorical')

# The input is going into two base models.
input_layer = Input(shape=(350, 350, 3))

x1 = DenseNet121(weights = 'imagenet',
                  include_top = False,
                  input_shape = (350, 350, 3))(input_layer)
x1 = Flatten()(x1)

x2 = Xception(weights = 'imagenet',
                  include_top = False,
                  input_shape = (350, 350, 3))(input_layer)
x2 = Flatten()(x2)
# concatenating the features that are extracted from two base models.
output = concatenate([x1, x2], axis = 1, name='Mohsen_net')

model_new = Model(input_layer, output, name='Concatenated')

conv_base = model_new

model = Sequential()
model.add(conv_base)
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dense(np.array(breeds).shape[0], activation = 'softmax'))
model.summary()
conv_base.trainable = False


epochs=60
learning_rate = 0.0001
decay_rate = learning_rate / epochs
momentum = 0.8

def exp_decay(epoch):
    lrate = learning_rate * np.exp(-decay_rate*epoch)
    return lrate

lr_rate = LearningRateScheduler(exp_decay)
callbacks_list = [lr_rate]


optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['acc'])


history = model.fit_generator(
    train_generator,
    steps_per_epoch=40,
    callbacks=callbacks_list,
    epochs=60,
    validation_data=validation_generator,
    validation_steps=50)


Found 4680 images belonging to 120 classes.
Found 5542 images belonging to 120 classes.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Concatenated (Functional)   (None, 371712)            27898984  
                                                                 
 dense_4 (Dense)             (None, 512)               190317056 
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_5 (Dense)             (None, 256)               131328    
                                                                 
 batch_normalization_4 (Batc  (None, 256)              1024      
 hNormalization)                                                 
                                                                 
 dense_6 (Dense)             (No

C:\Users\Mohsen\AppData\Local\Temp\ipykernel_17056\2856454598.py:83: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/60
40/40 [==============================] - 31s 613ms/step - loss: 4.7704 - acc: 0.0562 - val_loss: 6.5384 - val_acc: 0.0425 - lr: 1.0000e-04
Epoch 2/60
40/40 [==============================] - 23s 565ms/step - loss: 4.0959 - acc: 0.1250 - val_loss: 3.7409 - val_acc: 0.1675 - lr: 1.0000e-04
Epoch 3/60
40/40 [==============================] - 23s 571ms/step - loss: 3.5572 - acc: 0.2516 - val_loss: 2.8202 - val_acc: 0.3750 - lr: 1.0000e-04
Epoch 4/60
40/40 [==============================] - 23s 575ms/step - loss: 3.0785 - acc: 0.3641 - val_loss: 2.2311 - val_acc: 0.4938 - lr: 1.0000e-04
Epoch 5/60
40/40 [==============================] - 23s 562ms/step - loss: 2.7184 - acc: 0.4281 - val_loss: 2.3346 - val_acc: 0.4613 - lr: 9.9999e-05
Epoch 6/60
40/40 [==============================] - 23s 580ms/step - loss: 2.5117 - acc: 0.4797 - val_loss: 1.9058 - val_acc: 0.5612 - lr: 9.9999e-05
Epoch 7/60
40/40 [==============================] - 24s 601ms/step - loss: 2.4336 - acc: 0.4828 - va

40/40 [==============================] - 23s 581ms/step - loss: 0.5012 - acc: 0.8781 - val_loss: 0.4785 - val_acc: 0.8587 - lr: 9.9991e-05
Epoch 56/60
40/40 [==============================] - 23s 584ms/step - loss: 0.5349 - acc: 0.8625 - val_loss: 0.4687 - val_acc: 0.8775 - lr: 9.9991e-05
Epoch 57/60
40/40 [==============================] - 23s 579ms/step - loss: 0.5053 - acc: 0.8877 - val_loss: 0.4395 - val_acc: 0.8763 - lr: 9.9991e-05
Epoch 58/60
40/40 [==============================] - 23s 584ms/step - loss: 0.4557 - acc: 0.8828 - val_loss: 0.4134 - val_acc: 0.8763 - lr: 9.9991e-05
Epoch 59/60
40/40 [==============================] - 23s 586ms/step - loss: 0.4741 - acc: 0.8875 - val_loss: 0.3960 - val_acc: 0.8788 - lr: 9.9990e-05
Epoch 60/60
40/40 [==============================] - 23s 577ms/step - loss: 0.5457 - acc: 0.8672 - val_loss: 0.4797 - val_acc: 0.8438 - lr: 9.9990e-05


# 4- Fine Tunning

In [12]:
# Some part of the base model for transfer learning is going to be trained with our dataset.
# The first layers are still freezed, but the last layers of the base model is trainable to be fit to the new dataset.
# importing libraries
from tensorflow.keras.applications import InceptionResNetV2, Xception, NASNetMobile, MobileNetV2, DenseNet121, DenseNet169
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest') 
test_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory(copy_dir,
                                                    target_size = (224, 224),
                                                    batch_size = 20,
                                                    class_mode = 'categorical')
validation_generator = test_datagen.flow_from_directory(val_dir,
                                                        target_size = (224, 224),
                                                        batch_size = 20,
                                                        class_mode = 'categorical')
conv_base = NASNetMobile(weights = 'imagenet',
                  include_top = False,
                  input_shape = (224, 224, 3))
model = Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(512, activation = 'relu'))
model.add(layers.Dropout(0.1))
model.add(layers.Dense(256, activation = 'relu'))
model.add(layers.Dense(np.array(breeds).shape[0], activation = 'softmax'))
model.summary()
# creating a basis that has freezed starting layers, and trainable last layers of the base convolutional model.
conv_base.trainable = True
being_trainable = False
for layers_t in conv_base.layers:
    if layers_t.name == 'adjust_conv_projection_8' :
        being_trainable = True
    layers_t.trainable = being_trainable

optimizer = tf.keras.optimizers.Adam(0.0001)


model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy',
              metrics = ['acc'])


history = model.fit_generator(
    train_generator,
    steps_per_epoch=40,
    epochs=80,
    validation_data=validation_generator,
    validation_steps=50)

Found 4680 images belonging to 120 classes.
Found 5542 images belonging to 120 classes.
Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 NASNet (Functional)         (None, 7, 7, 1056)        4269716   
                                                                 
 flatten_4 (Flatten)         (None, 51744)             0         
                                                                 
 dense_7 (Dense)             (None, 512)               26493440  
                                                                 
 dropout_4 (Dropout)         (None, 512)               0         
                                                                 
 dense_8 (Dense)             (None, 256)               131328    
                                                                 
 dense_9 (Dense)             (None, 120)               30840     
                                

C:\Users\Mohsen\AppData\Local\Temp\ipykernel_17056\4277022785.py:55: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/80
40/40 [==============================] - 25s 359ms/step - loss: 4.7213 - acc: 0.0913 - val_loss: 3.6209 - val_acc: 0.2770
Epoch 2/80
40/40 [==============================] - 11s 280ms/step - loss: 3.6081 - acc: 0.2375 - val_loss: 2.2915 - val_acc: 0.4920
Epoch 3/80
40/40 [==============================] - 11s 280ms/step - loss: 2.7318 - acc: 0.3900 - val_loss: 1.5726 - val_acc: 0.6190
Epoch 4/80
40/40 [==============================] - 11s 287ms/step - loss: 2.1262 - acc: 0.5063 - val_loss: 1.3839 - val_acc: 0.6420
Epoch 5/80
40/40 [==============================] - 12s 289ms/step - loss: 1.8734 - acc: 0.5375 - val_loss: 1.1855 - val_acc: 0.6930
Epoch 6/80
40/40 [==============================] - 11s 285ms/step - loss: 1.7879 - acc: 0.5738 - val_loss: 1.3476 - val_acc: 0.6810
Epoch 7/80
40/40 [==============================] - 12s 299ms/step - loss: 1.6625 - acc: 0.5950 - val_loss: 1.2122 - val_acc: 0.6900
Epoch 8/80
40/40 [==============================] - 12s 290ms/step - 

40/40 [==============================] - 12s 294ms/step - loss: 0.4415 - acc: 0.8562 - val_loss: 1.6167 - val_acc: 0.6540
Epoch 63/80
40/40 [==============================] - 12s 289ms/step - loss: 0.4457 - acc: 0.8712 - val_loss: 1.5251 - val_acc: 0.6830
Epoch 64/80
40/40 [==============================] - 12s 288ms/step - loss: 0.4900 - acc: 0.8600 - val_loss: 1.6951 - val_acc: 0.6640
Epoch 65/80
40/40 [==============================] - 11s 284ms/step - loss: 0.5139 - acc: 0.8500 - val_loss: 1.5867 - val_acc: 0.6840
Epoch 66/80
40/40 [==============================] - 11s 282ms/step - loss: 0.4624 - acc: 0.8650 - val_loss: 1.6578 - val_acc: 0.6660
Epoch 67/80
40/40 [==============================] - 11s 278ms/step - loss: 0.4221 - acc: 0.8737 - val_loss: 1.3526 - val_acc: 0.7210
Epoch 68/80
40/40 [==============================] - 11s 284ms/step - loss: 0.3342 - acc: 0.8913 - val_loss: 1.4482 - val_acc: 0.6900
Epoch 69/80
40/40 [==============================] - 11s 282ms/step - loss